In [1]:
import os

os.chdir("../../")
%pwd

'c:\\Users\\anfe1\\OneDrive\\Escritorio\\Instaleap\\Instamarket'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    metrics_file_name: Path

In [3]:
from instamarket.constants import CONFIG_FILE_PATH
from instamarket.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        config_file_path = CONFIG_FILE_PATH

        self.config = read_yaml(config_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_path= config.model_path,
            metrics_file_name= config.metrics_file_name
        )
        
        return model_evaluation_config

In [34]:
import os
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from instamarket.utils.common import load_object
from instamarket.logging import logger

class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig) -> None:
        self.config = config

    @staticmethod
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def evaluate(self):
        logger.info("Loading test data")
        test_arr = load_object(os.path.join(self.config.data_path,"test.pkl"))

        logger.info("Split data in Independent & dependent variable")
        X_test,y_test = (test_arr.tocsc()[:,:-2], test_arr.tocsc()[:,-2:])

        logger.info("Loading model")
        model = load_object(self.config.model_path)

        logger.info("Start model evaluation")
        preds = model.predict(X_test)
        
        start_metrics = self.eval_metrics(y_test.toarray()[:, 0], preds[:, 0])
        logger.info("start_delay")
        logger.info(f"rmse: {start_metrics[0]}, mae: {start_metrics[0]}, r2: {start_metrics[1]}")

        end_metrics = self.eval_metrics(y_test.toarray()[:, 1], preds[:, 1])
        logger.info("end_delay")
        logger.info(f"rmse: {end_metrics[0]}, mae: {end_metrics[0]}, r2: {end_metrics[1]}")
        


In [35]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2024-04-20 16:36:03,800] 29 common - INFO - yaml file config\config.yml loaded successfully
[2024-04-20 16:36:03,800] 47 common - INFO - Created directory at: artifacts
[2024-04-20 16:36:03,800] 47 common - INFO - Created directory at: artifacts/model_evaluation
[2024-04-20 16:36:03,800] 21 1115185333 - INFO - Loading test data
[2024-04-20 16:36:03,817] 24 1115185333 - INFO - Split data in Independent & dependent variable
[2024-04-20 16:36:03,863] 27 1115185333 - INFO - Loading model
[2024-04-20 16:36:03,917] 30 1115185333 - INFO - Start model evaluation
[2024-04-20 16:36:04,225] 34 1115185333 - INFO - start_delay
[2024-04-20 16:36:04,225] 35 1115185333 - INFO - rmse: 83.75558886366154, mae: 83.75558886366154, r2: 47.141205323252734
[2024-04-20 16:36:04,225] 38 1115185333 - INFO - end_delay
[2024-04-20 16:36:04,225] 39 1115185333 - INFO - rmse: 84.62899796375187, mae: 84.62899796375187, r2: 48.72403077177631
